# Loading rut5 model

In [9]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

In [10]:
!pip install sentencepiece

In [11]:
from_pkl_file = True

tokenizer_file = 't5_tokenizer.pkl'
model_file = 't5_model.pkl'

In [12]:
import pickle

if from_pkl_file:
    with open(tokenizer_file, 'rb') as f:
        tokenizer = pickle.load(f)
else:        
    tokenizer = T5Tokenizer.from_pretrained(
        'cointegrated/rut5-small-chitchat2'
    )
    with open(tokenizer_file, 'wb') as f:
        pickle.dump(tokenizer, f)

In [13]:
if from_pkl_file:
    with open(model_file, 'rb') as f:
        model = pickle.load(f)
else:        
    model = T5ForConditionalGeneration.from_pretrained('cointegrated/rut5-small-chitchat2')
    with open(model_file, 'wb') as f:
        pickle.dump(model, f)

In [14]:
model: T5ForConditionalGeneration

# Test run

In [22]:
text = 'Привет! Расскажи, как твои дела?'
inputs = tokenizer(text, return_tensors='pt')

In [23]:
with torch.no_grad():
    # https://huggingface.co/docs/transformers/main/en/main_classes/text_generation
    hypotheses = model.generate(
        **inputs,
        temperature=0.9,
        do_sample=True,  # sampling or greedy decoding 
        # (at each decoding step selects the token with the highest prob without considering the impact on future tokens)
        top_p=0.7,
        num_return_sequences=3,
        repetition_penalty=2.5,  # https://arxiv.org/pdf/1909.05858.pdf
        max_length=32,
    )

In [24]:
for h in hypotheses:
    print(tokenizer.decode(h, skip_special_tokens=True))

Что это?
Это что?
Мне нравится.


# Dialogue loop

In [45]:
import time

user_prefix = '<user>'
bot_prefix = '<bot>'
user_input = input('Talk to bot<3')
history = ''
verbose_history = []
while user_input != 'q':
    user_input = user_prefix + user_input
    history += user_input
    inputs = tokenizer(history, return_tensors='pt')
    with torch.no_grad():
        hypotheses = model.generate(
            **inputs,
            temperature=0.9,
            do_sample=True, 
            top_p=0.7,
            num_return_sequences=5,
            repetition_penalty=2.5,
            max_length=32,
        )
        bot_utterances = []
        for h in hypotheses:
            bot_utterances.append(tokenizer.decode(h, skip_special_tokens=True))
        history += bot_prefix + bot_utterances[0]
        print(bot_utterances)
        verbose_history.append((user_input, bot_prefix + bot_utterances[0]))
    time.sleep(1)
    user_input = input('Talk to bot<3')

print('history:', history)    
print('verbose history:')
for user, bot in verbose_history:
    print(user)
    print(bot)

['Это так?', 'Что это?', 'Что это такое?', 'Что это?', 'Что это такое?']
['Это так!', 'Что это такое?', 'Что это?', 'Что это такое?', 'Это так, но ты самый.']
['Что это такое?', 'Что это такое?', 'Это так!', 'Это так, как в первый раз?', 'Что это такое?']
['Это так, это ты', 'Это так!', 'Это это так', 'Это тот, что ты имеешь ввиду.', 'Это такая вещь, которую ты хотела сказать...']
['Это так.', 'Это ты, миа...', 'Это тот, который сейчас делает.', 'Что это такое?', 'Это ты такой!']
history: <user>я Дима<bot>Это так?<user>кто я<bot>Это так!<user>как меня зовут<bot>Что это такое?<user>КТО Я<bot>Это так, это ты<user>КАКОЕ ИМЯ У МЕНЯ<bot>Это так.
verbose history:
<user>я Дима
<bot>Это так?
<user>кто я
<bot>Это так!
<user>как меня зовут
<bot>Что это такое?
<user>КТО Я
<bot>Это так, это ты
<user>КАКОЕ ИМЯ У МЕНЯ
<bot>Это так.
